Adapted from: https://github.com/phillipi/pix2pix

In [1]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

Cloning into 'pytorch-CycleGAN-and-pix2pix'...
remote: Enumerating objects: 2443, done.
remote: Total 2443 (delta 0), reused 0 (delta 0), pack-reused 2443
Receiving objects: 100% (2443/2443), 8.13 MiB | 28.13 MiB/s, done.
Resolving deltas: 100% (1570/1570), done.


In [ ]:
import os
os.chdir('pytorch-CycleGAN-and-pix2pix/')

In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 676 kB 19.9 MB/s 
     |████████████████████████████████| 1.7 MB 58.7 MB/s 
     |████████████████████████████████| 52 kB 1.8 MB/s 
     |████████████████████████████████| 97 kB 9.2 MB/s 
     |████████████████████████████████| 180 kB 68.0 MB/s 
     |████████████████████████████████| 140 kB 69.1 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-py3-none-any.whl size=655250 sha256=f546c69af7f5c4ca7efdfb4ca92f43c2df3a960eb1f1bdcaef438d4760f9f414
  Stored in directory: /root/.cache/pip/wheels/2d/d1/9b/cde923274eac9cbb6ff0d8c7c72fe30a3da9095a38fd50bbf1
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=a38e07861d5a140017f5dfd507ac8a1c2f6fbbe6b740945b331526725cdea5dc
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl

I pair the original highway images with my hand-drawn masks

In [ ]:
!python datasets/combine_A_and_B.py --fold_A /content/drive/MyDrive/BayAreaHighway250Raw_pix --fold_B /content/drive/MyDrive/BayAreaHighway250Mask2 --fold_AB /content/drive/MyDrive/BayAreaHighway250AB_pix

[fold_A] =  /content/drive/MyDrive/BayAreaHighway250Raw_pix
[fold_B] =  /content/drive/MyDrive/BayAreaHighway250Mask2
[fold_AB] =  /content/drive/MyDrive/BayAreaHighway250AB_pix
[num_imgs] =  1000000
[use_AB] =  False
[no_multiprocessing] =  False
split = train, use 100/100 images
split = train, number of images = 100


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


I install the pre-trained model to improve on

In [ ]:
!bash ./scripts/download_pix2pix_model.sh sat2map

Note: available models are edges2shoes, sat2map, map2sat, facades_label2photo, and day2night
Specified [sat2map]
for details.

--2021-12-02 09:21:55--  http://efrosgans.eecs.berkeley.edu/pix2pix/models-pytorch/sat2map.pth
Resolving efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)... 128.32.244.190
Connecting to efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)|128.32.244.190|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217704720 (208M)
Saving to: ‘./checkpoints/sat2map_pretrained/latest_net_G.pth’

./checkpoints/sat2m 100%[===================>] 207.62M  3.26MB/s    in 54s     

2021-12-02 09:22:49 (3.83 MB/s) - ‘./checkpoints/sat2map_pretrained/latest_net_G.pth’ saved [217704720/217704720]



I finetune the model with the data pairs

In [ ]:
!python train.py --dataroot /content/drive/MyDrive/BayAreaHighway250AB_pix --name sat2map_pretrained --model pix2pix

----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           
                 dataroot: /content/drive/MyDrive/BayAreaHighway250AB_pix	[default: None]
             dataset_mode: aligned                       
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: 1                             
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                    epoch: latest                       

# Testing

-   `python test.py --dataroot ./datasets/facades --direction BtoA --model pix2pix --name facades_pix2pix`

Change the `--dataroot`, `--name`, and `--direction` to be consistent with your trained model's configuration and how you want to transform images.

> from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix:
> Note that we specified --direction BtoA as Facades dataset's A to B direction is photos to labels.

> If you would like to apply a pre-trained model to a collection of input images (rather than image pairs), please use --model test option. See ./scripts/test_single.sh for how to apply a model to Facade label maps (stored in the directory facades/testB).

> See a list of currently available models at ./scripts/download_pix2pix_model.sh

I pair the test set to create the test set masks

In [ ]:
!python datasets/combine_A_and_B.py --fold_A /content/drive/MyDrive/NOLATest --fold_B /content/drive/MyDrive/NOLATest --fold_AB /content/drive/MyDrive/NOLATestAB

[fold_A] =  /content/drive/MyDrive/NOLATest
[fold_B] =  /content/drive/MyDrive/NOLATest
[fold_AB] =  /content/drive/MyDrive/NOLATestAB
[num_imgs] =  1000000
[use_AB] =  False
[no_multiprocessing] =  False
split = test, use 5/5 images
split = test, number of images = 5


I run inference on the test set. I would change this to run inference on 250 unpaired highway images to create the training masks for the "full model" configuration. Within utils I edit:     




```
def save_images(webpage, visuals, image_path, aspect_ratio=1.0, width=256, use_wandb=False):
    """Save images to the disk.

    Parameters:
        webpage (the HTML class) -- the HTML webpage class that stores these imaegs (see html.py for more details)
        visuals (OrderedDict)    -- an ordered dictionary that stores (name, images (either tensor or numpy) ) pairs
        image_path (str)         -- the string is used to create image paths
        aspect_ratio (float)     -- the aspect ratio of saved images
        width (int)              -- the images will be resized to width x width

    This function will save images stored in 'visuals' to the HTML file specified by 'webpage'.
    """
    image_dir = webpage.get_image_dir()
    short_path = ntpath.basename(image_path[0])
    name = os.path.splitext(short_path)[0]

    webpage.add_header(name)
    ims, txts, links = [], [], []
    ims_dict = {}
    for label, im_data in visuals.items():
        im = util.tensor2im(im_data)
        image_name = '%s_%s.png' % (name, label)
        save_path = os.path.join(image_dir, image_name)
        util.save_image(im, save_path, aspect_ratio=aspect_ratio)
        ims.append(image_name)
        txts.append(label)
        links.append(image_name)
        if use_wandb:
            ims_dict[label] = wandb.Image(im)
    webpage.add_images(ims, txts, links, width=width)
    if use_wandb:
        wandb.log(ims_dict)
```

With



```
def save_images(webpage, visuals, image_path, aspect_ratio=1.0, width=256, use_wandb=False):
    """Save images to the disk.

    Parameters:
        webpage (the HTML class) -- the HTML webpage class that stores these imaegs (see html.py for more details)
        visuals (OrderedDict)    -- an ordered dictionary that stores (name, images (either tensor or numpy) ) pairs
        image_path (str)         -- the string is used to create image paths
        aspect_ratio (float)     -- the aspect ratio of saved images
        width (int)              -- the images will be resized to width x width

    This function will save images stored in 'visuals' to the HTML file specified by 'webpage'.
    """
    image_dir = webpage.get_image_dir()
    short_path = ntpath.basename(image_path[0])
    name = os.path.splitext(short_path)[0]

    webpage.add_header(name)
    ims, txts, links = [], [], []
    ims_dict = {}
    for label, im_data in visuals.items():
      if label == "fake_B"
        im = util.tensor2im(im_data)
        image_name = '%s_%s.png' % (name, label)
        save_path = os.path.join(image_dir, image_name)
        util.save_image(im, save_path, aspect_ratio=aspect_ratio)
        ims.append(image_name)
        txts.append(label)
        links.append(image_name)
        if use_wandb:
            ims_dict[label] = wandb.Image(im)
    webpage.add_images(ims, txts, links, width=width)
    if use_wandb:
        wandb.log(ims_dict)
```

To create cleaner folders with only the test masks I need


In [ ]:
!python test.py --dataroot /content/drive/MyDrive/NOLATestAB --model pix2pix --name sat2map_pretrained --checkpoints_dir /content/drive/MyDrive/checkpoints

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: /content/drive/MyDrive/checkpoints	[default: ./checkpoints]
                crop_size: 256                           
                 dataroot: /content/drive/MyDrive/NOLATestAB	[default: None]
             dataset_mode: aligned                       
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load_ite